In [ ]:
# Importamos la librería 'requests', que nos permite hacer peticiones a servicios web.
import requests

# Importamos 'json', que nos ayuda a manejar datos en formato JSON.
import json

# Desde IPython, traemos una forma de mostrar datos JSON.
from IPython.display import JSON

# Importamos todas las claves o credenciales guardadas en el archivo 'keys'.
from keys import *

In [ ]:
# Solo se permiten 200 consultas al mes

# URL del servicio que da info de vuelos en el aeropuerto EGLL (Heathrow) en ese rango de horas
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EGLL/2025-09-01T08:00/2025-09-01T20:00"

# Parámetros para filtrar: llegadas, vuelos cancelados, compartidos, de carga, privados, etc.
querystring = {
    "withLeg": "true",
    "direction": "Arrival",
    "withCancelled": "true",
    "withCodeshared": "true",
    "withCargo": "true",
    "withPrivate": "true",
    "withLocation": "false"
}

# Encabezados necesarios para acceder al API, incluyendo la clave
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
}

# Se hace la petición GET al API con la URL, encabezados y parámetros
response = requests.request("GET", url, headers=headers, params=querystring)


In [ ]:
# Si la respuesta del API fue exitosa (código 200)
if response.status_code == 200:
    new_data = response.json()  # Convertimos la respuesta en formato JSON

    try:
        # Intentamos abrir el archivo donde guardamos datos anteriores
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)  # Cargamos los datos ya guardados
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []  # Si no existe o está vacío, empezamos con una lista nueva

    existing_data.append(new_data)  # Agregamos los nuevos datos a la lista

    # Guardamos todo (viejo + nuevo) en el archivo
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")  # Confirmamos que se guardó bien
else:
    # Si hubo error en la respuesta, mostramos el código de error
    print("Failed to retrieve data from the API. Status code:", response.status_code)


Data appended to data.json file.


In [ ]:
# Guardamos la respuesta del API en formato JSON
flight_arrivals = response.json()

# Recorremos cada vuelo en la lista 
for flight in flight_arrivals:
    # Hora programada de llegada
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    
    # Número de vuelo
    flight_arrivals['arrivals'][0]['number']
    
    # Terminal de llegada
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    
    # Nombre del aeropuerto de origen
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    
    # Modelo del avión
    flight_arrivals['arrivals'][0]['aircraft']['model']
    
    # Nombre de la aerolínea
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
# Importamos pandas para manejar datos en tablas (DataFrames)
import pandas as pd

# Importamos 'nan' por si necesitamos marcar valores vacíos más adelante
from numpy import nan

# Convertimos la respuesta del API en formato JSON
flight_arrivals = response.json()

# Función para extraer y organizar los datos de cada vuelo
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],  # Hora programada de llegada
        'flight_number': flights['number'],                             # Número de vuelo
        'from': flights['departure']['airport']['name'],                # Aeropuerto de origen
        'airline': flights['airline']['name'],                          # Nombre de la aerolínea
        'aircraft': flights['aircraft']['model']                        # Modelo del avión
    })
    return flights_df  # Devuelve los datos como tabla

# Aplicamos la función a cada vuelo y juntamos todo en un solo DataFrame
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

# Mostramos las primeras filas para revisar que todo esté bien
flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,MH 9899,Toulouse-Blagnac,Malaysia,Airbus A319,2025-09-01 07:20Z,2025-09-01 08:20+01:00
0,QR 5970,Toulouse-Blagnac,Qatar,Airbus A319,2025-09-01 07:20Z,2025-09-01 08:20+01:00
0,BA 377,Toulouse-Blagnac,British,Airbus A319,2025-09-01 07:20Z,2025-09-01 08:20+01:00
0,QF 3510,Toulouse-Blagnac,Qantas,Airbus A319,2025-09-01 07:20Z,2025-09-01 08:20+01:00
0,AA 6798,Toulouse-Blagnac,American,Airbus A319,2025-09-01 07:20Z,2025-09-01 08:20+01:00
